## Remote 4Dstem data acquisition using Autoscript

## 1. Import 

In [ ]:
# the Autoscript packages

from autoscript_tem_microscope_client import TemMicroscopeClient
from autoscript_tem_microscope_client.enumerations import *
from autoscript_tem_microscope_client.structures import *
# General packages
import os, time, sys, math

# General image processing packages
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import cv2 as cv

os.makedirs('plots', exist_ok=True)# directory to store plots
os.makedirs('data', exist_ok=True)# directory to store data



In [ ]:
microscope = TemMicroscopeClient()
ip = ""
if ip == "":
    ip = input("Please enter the IP address of the microscope: ")
microscope.connect(ip)
print("Connected to the microscope")

## 2. Utility Functions

In [ ]:
# query position:
def print_mic_stage_position():
    position = microscope.specimen.stage.position
    pos_dict = {"x": position.x, "y": position.y, "z": position.z, "alpha": position.a, "beta": position.b}
    print(pos_dict)

def get_pixel_size(HAADF):
    pixelsize_x, pixelsize_y = HAADF.metadata.binary_result.pixel_size.x, HAADF.metadata.binary_result.pixel_size.y
    # xmove, ymove = displacement[0]*pixelsize_x, displacement[1]*pixelsize_y
    print(pixelsize_x, pixelsize_y)
    return pixelsize_x, pixelsize_y

def plot_acquisition_return_array(image):
    img = image.data - np.min(image.data)
    image_data = (255*(img/np.max(img))).astype(np.uint8)

    # # plot the image
    # fig = plt.figure(figsize=(6,6))
    # plt.imshow(image_data, cmap='gray')
    # plt.title('Acquired image')
    return image_data

def print_beam_pos():
    microscope.optics.deflectors.beam_shift
    
# def move_scope(x, y, )

## 3. Workflow

In [ ]:
print_mic_stage_position()
haadf_image = microscope.acquisition.acquire_stem_image(DetectorType.HAADF, 128, 4e-6)# haadf is pixel wise
haadf_image_data = plot_acquisition_return_array(haadf_image)
print("Displaying image after acquisition and saving to npz file")
np.savez("data/test_image_128_3.npz", haadf_image_data )
plt.imshow(haadf_image_data)

In [ ]:
# Get pixel size information from the HAADF image
pixelsize_x, pixelsize_y = haadf_image.metadata.binary_result.pixel_size.x, haadf_image.metadata.binary_result.pixel_size.y


In [ ]:
# Define dimensions for HAADF and CETA
haadf_dim = haadf_image_data.shape
ceta_dim = (512, 512)

# Initialize arrays with zeros
# stem4dBF: Placeholder for Bright Field STEM data
stem4dBF = np.zeros_like(haadf_image_data)

# stem4d: 4D array for storing CETA camera images
stem4d = np.zeros(haadf_dim + ceta_dim, dtype=np.float64)

# Reset beam shift to origin (0, 0)
microscope.optics.deflectors.beam_shift = [0, 0]

# Loop over the pixel grid
# tqdm provides a progress bar for the loops
for x in tqdm(range(int(haadf_dim[0])), desc='Y Progress'):
    dmove_x = pixelsize_x * 1
    old_x, old_y = microscope.optics.deflectors.beam_shift 
    # Update beam shift in the x direction only
    microscope.optics.deflectors.beam_shift = [old_x + dmove_x, old_y]
    for y in tqdm(range(int(haadf_dim[1])), desc='X Progress', leave=False):
        dmove_y = pixelsize_y * 1
        old_x, old_y = microscope.optics.deflectors.beam_shift 
        print("old beam_pos", microscope.optics.deflectors.beam_shift)
        # Update beam shift in the y direction only
        microscope.optics.deflectors.beam_shift = [old_x, old_y + dmove_y]
        print("new beam_pos", microscope.optics.deflectors.beam_shift)

        # Acquire CETA camera image
        ceta = microscope.acquisition.acquire_camera_image(CameraType.BM_CETA, 512, 25e-4) # 25 ms exposure time
        ceta_data = plot_acquisition_return_array(ceta)
        new_x, new_y = microscope.optics.deflectors.beam_shift 


        # Update data arrays with the acquired image and its mean value
        stem4dBF[x][y] = ceta_data.mean()
        stem4d[x][y] = ceta_data

        # Plot both stem4dBF and the current CETA image side by side
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))

        # Display the mean image of stem4dBF
        fig.suptitle(f"STEM4D Bright Field and CETA Image at pixel positionx: {x}, y: {y}")
        axs[0].imshow(stem4dBF)
        axs[0].set_title("stem4dBF - Mean of the CETA image")

        # Display the current CETA image
        axs[1].imshow(ceta_data)
        axs[1].set_title(f"CETA image at x: {new_x}, y: {new_y}")

        plt.savefig(f"plots/stem4dBF_{x}_{y}.png")


# 4. Sparse sampling

In [ ]:
print_mic_stage_position()
haadf_image = microscope.acquisition.acquire_stem_image(DetectorType.HAADF, 128, 4e-6)# haadf is pixel wise
haadf_image_data = plot_acquisition_return_array(haadf_image)
print("Displaying image after acquisition and saving to npz file")
np.savez("data/test_image_128_3.npz", haadf_image_data )
plt.imshow(haadf_image_data)

In [ ]:


# Define dimensions for HAADF and CETA
haadf_dim = haadf_image_data.shape
ceta_dim = (512, 512)

# Initialize arrays with zeros
# stem4dBF: Placeholder for Bright Field STEM data
stem4dBF = np.zeros_like(haadf_image_data)

# stem4d: 4D array for storing CETA camera images
stem4d = np.zeros(haadf_dim + ceta_dim, dtype=np.float64)

# Reset beam shift to origin (0, 0)
microscope.optics.deflectors.beam_shift = [0, 0]

# Loop over the pixel grid
for x in tqdm(range(int(haadf_dim[0])), desc='Y Progress'):
    dmove_x = pixelsize_x * 1
    old_x, old_y = microscope.optics.deflectors.beam_shift 
    # Update beam shift in the x direction only
    microscope.optics.deflectors.beam_shift = [old_x + dmove_x, old_y]
    step_size_y = 5# scaling factor for y
    for y in tqdm(range(int(int(haadf_dim[1])/step_size_y)), desc='X Progress', leave=False):
        dmove_y = pixelsize_y * step_size_y
        old_x, old_y = microscope.optics.deflectors.beam_shift 
        print("old beam_pos", microscope.optics.deflectors.beam_shift)
        # Update beam shift in the y direction only
        microscope.optics.deflectors.beam_shift = [old_x, old_y + dmove_y]
        print("new beam_pos", microscope.optics.deflectors.beam_shift)

        # Acquire CETA camera image
        ceta = microscope.acquisition.acquire_camera_image(CameraType.BM_CETA, 512, 25e-4) # 25 ms exposure time
        ceta_data = plot_acquisition_return_array(ceta)
        new_x, new_y = microscope.optics.deflectors.beam_shift 

        # Update data arrays with the acquired image and its mean value
        stem4dBF[x][y*step_size_y] = ceta_data.mean()
        stem4d[x][y*step_size_y] = ceta_data

        # Plot both stem4dBF and the current CETA image side by side
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))

        # Display the mean image of stem4dBF
        fig.suptitle(f"STEM4D Bright Field and CETA Image at pixel positionx: {x}, y: {y}")
        axs[0].imshow(stem4dBF)
        axs[0].set_title("stem4dBF - Mean of the CETA image")

        # Display the current CETA image
        axs[1].imshow(ceta_data)
        axs[1].set_title(f"CETA image at x: {new_x}, y: {new_y}")

        plt.savefig(f"plots/stem4dBF_{x}_{y}.png")
